In [ ]:
# import numpy as np
# import pandas as pd
# import xarray as xr
# from glob import glob
# import os
# from pathlib import Path

In [ ]:
# glider = "ng645-20210613T0000"
# days_offset = 2

In [ ]:
# # Set main path of data and plot location
# root_dir = Path.home() / "Documents"

# # Paths to data sources
# path_data = (root_dir / "data") # create data path
# path_rtofs = "https://tds.marine.rutgers.edu/thredds/dodsC/cool/rtofs/rtofs_us_east_scraped"
# path_gliders = (path_data / "gliders")
# path_impact = (path_data / "impact_metrics")
# path_impact_calculated = path_impact / "calculated"
# path_impact_model = path_impact / "models"

In [ ]:
# # Read glider dataframe output from erddap
# glider_pickle = path_gliders / f"{glider}_data.pkl"

# try:
#     df = pd.read_pickle(glider_pickle)
# except FileNotFoundError:
#     # Download glider data from erddap with dataset id
#     df = get_glider_by_id(glider)
#     df.to_pickle(glider_pickle) # Save glider data to pickle file
    
# df = df.reset_index()
# df = df.rename({
#     "time (UTC)": "time",
#     "longitude (degrees_east)": "lon",
#     "latitude (degrees_north)": "lat",
#     "pressure (decibar)": "pressure",
#     "temperature (degrees_C)": "temp",
#     "depth (m)": "depth",
#     "salinity (1)": "salinity",
#     "conductivity (mS cm-1)": "conductivity",
#     "density (kg m-3)": "density",
# }, axis=1)

# df = df.set_index("time").sort_index()
# t0 = df.index.min().strftime("%Y-%m-%d")
# t1 = df.index.max().strftime("%Y-%m-%d")

In [ ]:
# df.head(50)

In [ ]:
# from hurricanes.calc import depth_bin, depth_interpolate
# interp = df.reset_index().groupby(['time']).apply(depth_interpolate, index=['depth'])
# interp

In [ ]:
# ds = interp.to_xarray()
# ds

In [ ]:
# resampled = ds.resample(time='6H').interpolate('linear')
# resampled

In [ ]:
# resampled.to_netcdf('/Users/mikesmith/Documents/ng645_resampled_6h.nc')

In [ ]:
# resampled = xr.open_dataset('/Users/mikesmith/Documents/data/gliders/ng645_resampled_6h.nc')
# resampled

In [ ]:
# import matplotlib.pyplot as plt
# from cmocean import cm

# resampled.salinity.plot(x='time', y='depth', figsize=(20,8), cmap=cm.haline)
# plt.ylim([500, 0])

In [ ]:
# resampled.temp.plot(x='time', y='depth', figsize=(20,8), cmap=cm.thermal)
# plt.ylim([500, 0])

In [ ]:
# resampled.density.plot(x='time', y='depth', figsize=(20,8), cmap=cm.dense)
# plt.ylim([500, 0])

In [ ]:
# gtime = []
# glon = []
# glat = []

# for time in resampled.time:
#     tds = resampled.sel(time=time)
#     gtime.append(pd.to_datetime(tds.time.values))
#     glon.append(np.unique(tds.lon.values)[0])
#     glat.append(np.unique(tds.lat.values)[0])

In [ ]:
# zipped = list(zip(gtime, glon, glat))

In [ ]:
# tmp = pd.DataFrame(zipped, columns=["time", "lon", "lat"]).dropna()
# tmp

In [ ]:
# tmp = tmp.iloc[::40, :]

In [ ]:
# tmp['lon'] = np.mod(tmp['lon']-180, 360)+180

In [ ]:
# tmp

In [ ]:
# gtime = tmp['time'].tolist()
# glon = tmp['lon'].tolist()
# glat = tmp['lat'].tolist()

In [ ]:
# # RTOFS
# url = "https://tds.marine.rutgers.edu/thredds/dodsC/cool/rtofs/rtofs_us_east_scraped"
# rtofs = xr.open_dataset(url)

# # url = "/Users/mikesmith/Documents/data/rtofs_2021.zarr"
# # rtofs = xr.open_zarr(url)

# # Save rtofs lon and lat as variables to speed up indexing calculation
# rtofs_lon = rtofs.lon.values
# rtofs_lat = rtofs.lat.values

# # Find index of nearest lon and lat points
# calc_lon = np.array([find_nearest(rtofs_lon[0, :], lon) for lon in glon])
# calc_lat = np.array([find_nearest(rtofs_lat[:, 0], lat) for lat in glat])
# # calc_lat

In [ ]:
# # Create dataarrays for pointwise indexing
# # https://stackoverflow.com/questions/40544846/
# # read-multiple-coordinates-with-xarray
# times = xr.DataArray(gtime, dims='point')
# lons = xr.DataArray(calc_lon[:,1], dims='point')
# lats = xr.DataArray(calc_lat[:,1], dims='point')

In [ ]:
# rtofs

In [ ]:
# grtofs = rtofs[['temperature', 'salinity']].sel(
#     time=times, 
#     y=lats,
#     x=lons,
#     method='nearest'
# )

In [ ]:
# grtofs

In [ ]:
# %time grtofs.load()

In [ ]:
# grtofs.to_netcdf("/Users/mikesmith/Documents/ng645_6h_rtofs_data.nc")

In [ ]:
# # Calculate upper ocean metrics from rtofs and add to DataFrame
# rtofs_df = pd.DataFrame()
# for t in trtofs.point:
#     temp = trtofs.sel(point=t)
#     rdf = calculate_upper_ocean_metrics(
#         pd.to_datetime(temp.time.values),
#         temp.temperature.values,
#         temp.salinity.values,
#         temp.depth.values,
#         np.full(temp.temperature.values.shape, temp.lat.values),
#         )
#     rtofs_df = pd.concat([rtofs_df, rdf])
#     rtofs_df.to_csv(impact_calculated_dir / f"{glider}_calculated_rtofs_data.csv")
# rtofs_df.to_pickle(impact_calculated_dir / f"{glider}_calculated_rtofs_data.pkl")

In [ ]:
# gtime

In [1]:
import xarray as xr
import datetime as dt

gtime = [dt.datetime(2021,6,13,6,0,0),
         dt.datetime(2021,6,23,6,0,0),
         dt.datetime(2021,7,3,6,0,0),
         dt.datetime(2021,7,13,6,0,0),
         dt.datetime(2021,7,23,6,0,0),
         dt.datetime(2021,8,2,6,0,0),
         dt.datetime(2021,8,12,6,0,0),
         dt.datetime(2021,8,22,6,0,0),
         dt.datetime(2021,9,1,6,0,0),
         dt.datetime(2021,9,11,6,0,0),
         dt.datetime(2021,9,21,6,0,0)
        ]
glon = [
    265.41368532029145,
    266.2228786882868, 
    267.2704577270577, 
    268.85892911601684, 
    269.18903847079787, 
    269.3574762286588, 
    270.30725702433676, 
    271.4906198819087, 
    272.21700585406757, 
    272.0912898757826, 
    271.76470859324746
]
glat = [
    27.638446297249928, 
    27.056615139362126, 
    27.15419373137167,
    27.40356144679907,
    27.43763012714433,
    26.666685239752848,
    27.774503201623297,
    28.2032588731131,
    28.11229420537534,
    28.651781244954552, 
    29.194729088837246
]

url = "https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0"
ds = xr.open_dataset(url, drop_variables='tau')
# ds.nbytes / 1e9 #123944.239138864
# ds

In [ ]:
# Subset the area surrounding the transect (Do I even need to do this?)
lons = [min(glon)-.1, max(glon)+.1]
lats = [min(glat)-.1, max(glat)+.1]

# only want to look at  the upper 1000 meters
tds = ds[['water_temp', 'salinity']].sel(
    time=slice(min(gtime), max(gtime)),
    depth=slice(0, 1000), 
    lon=slice(lons[0], lons[1]), 
    lat=slice(lats[0], lats[1])
)
# tds.nbytes / 1e9 # 1.251028936
tds

In [ ]:
# Select the time, lon, and lat nearest the transect
xds = tds.sel(
    time=xr.DataArray(gtime, dims='point'), 
    lon=xr.DataArray(glon, dims='point'),
    lat=xr.DataArray(glat, dims="point"),
    method='nearest'
)

In [ ]:
# xds.nbytes / 1e9 # 3.432e-06
xds # this hangs when I try to look at the result

# # from dask.diagnostics import ProgressBar
# # delayed = xds.to_netcdf('gofs_transect_test.nc', compute=False)

# # with ProgressBar():
#     # results = delayed.compute()

In [ ]:
# xds

In [2]:
# url of the thredds server containing scraped RTOFS data
url = "https://tds.marine.rutgers.edu/thredds/dodsC/cool/rtofs/rtofs_us_east_scraped"

# Use the xarray open_dataset function to access the dataset via opendap
ds = xr.open_dataset(url)
ds

<xarray.Dataset>
Dimensions:      (depth: 40, y: 1710, x: 742, time: 1632)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 1 2 3 4 5 6 7 ... 1704 1705 1706 1707 1708 1709 1710
  * x            (x) int32 1 2 3 4 5 6 7 8 9 ... 735 736 737 738 739 740 741 742
  * time         (time) datetime64[ns] 2021-03-04T06:00:00 ... 2022-04-19
Data variables:
    date         (time) float64 ...
    lat          (y, x) float32 ...
    lon          (y, x) float32 ...
    u            (time, depth, y, x) float32 ...
    v            (time, depth, y, x) float32 ...
    temperature  (time, depth, y, x) float32 ...
    salinity     (time, depth, y, x) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [3]:
def find_nearest(array, value):
    """
    Find the index of the closest value in an array

    Args:
        array (list or np.array): Array to locate values on
        value (list or np.array): Values to locate on array

    Returns:
        float, int: value(s), index of the nearest neighbors 
    """
    # Subtract values from array. Use .argmin() to find the indices of the minimum values along an axis.
    idx = (np.abs(array-value)).argmin()
    
    # Return the closest value and the index of the closest value.
    # Use .flat on the array to collapse the m x n array into a one dimensional array
    return array.flat[idx], idx

In [4]:
rtofs_lon = ds.lon.values
rtofs_lat = ds.lat.values

In [5]:
import numpy as np
rlon = np.mod(np.array(glon)+180, 306)-180
rlon

array([-40.58631468, -39.77712131, -38.72954227, -37.14107088,
       -36.81096153, -36.64252377, -35.69274298, -34.50938012,
       -33.78299415, -33.90871012, -34.23529141])

In [6]:
clon = []
clat = []
for ind, val in enumerate(rlon):
    # Find index of nearest lon and lat points
    _, lon_ind = find_nearest(rtofs_lon[0, :], rlon[ind])
    _, lat_ind = find_nearest(rtofs_lat[:, 0], glat[ind])
    clon.append(lon_ind)
    clat.append(lat_ind)

clon, clat

([741, 741, 741, 741, 741, 741, 741, 741, 741, 741, 741],
 [360, 352, 353, 356, 357, 346, 362, 368, 366, 374, 382])

In [9]:
tds = ds.isel(
#     time=xr.DataArray(gtime, dims='point'),
    x=xr.DataArray(clon, dims='point'),
    y=xr.DataArray(clat, dims='point'),
)

In [10]:
tds

<xarray.Dataset>
Dimensions:      (depth: 40, point: 11, time: 1632)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
    y            (point) int32 361 353 354 357 358 347 363 369 367 375 383
    x            (point) int32 742 742 742 742 742 742 742 742 742 742 742
  * time         (time) datetime64[ns] 2021-03-04T06:00:00 ... 2022-04-19
Dimensions without coordinates: point
Data variables:
    date         (time) float64 2.021e+07 2.021e+07 ... 2.022e+07 2.022e+07
    lat          (point) float32 27.66 27.09 27.16 27.37 ... 28.08 28.65 29.21
    lon          (point) float32 -40.72 -40.72 -40.72 ... -40.72 -40.72 -40.72
    u            (time, depth, point) float32 ...
    v            (time, depth, point) float32 ...
    temperature  (time, depth, point) float32 ...
    salinity     (time, depth, point) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [ ]:
from dask.diagnostics import ProgressBar
delayed = tds.to_netcdf('rtofs_transect_test.nc', compute=False)

In [ ]:
with ProgressBar():
    results = delayed.compute()